In [46]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model
from keras.layers import merge, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [47]:
from pathlib import Path
train_dir = "kaggle_bee_vs_wasp/"
bs = 64 # Batch size
resize_size = 224 # for training, resize all the images to a square of this size
training_subsample = 1 # for development, use a small fraction of the entire dataset rater than full dataset

bees_vs_wasps_dataset_path=Path(train_dir) # this is relative to the "example_notebook" folder. Modify this to reflect your setup
df_labels = pd.read_csv(bees_vs_wasps_dataset_path/'labels.csv')
df_labels=df_labels.set_index('id')
# perform dataset subsampling
df_labels = df_labels.sample(frac=training_subsample, axis=0)
insect_class = {'bee': 0,'wasp': 1, 'insect': 2,'other': 3} 
#df_labels.label = [insect_class[item] for item in df_labels.label] 
df_labels = df_labels[['path','label']]

In [48]:
#Uncomment to read data from csv
#df_labels = pd.read_csv("Dataset_full.csv") #full dataset
#df_labels = pd.read_csv("Dataset_10precent.csv") #0.1 fraction for development

In [49]:
train_df, test_df = train_test_split(df_labels, test_size=0.1)
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

In [50]:
image_input = Input(shape=(resize_size, resize_size, 3))
model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')
last_layer = model.get_layer('block5_pool')
last_output = last_layer.output
x = Flatten()(last_output)
custom_vgg_model = Model(image_input, x)
custom_vgg_model.compile(loss='sparse_categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
for layer in custom_vgg_model.layers:
	layer.trainable = False
custom_vgg_model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)     

In [51]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    train_dir,  
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=1
)


filenames = train_generator.filenames
n_train_samples = len(filenames)
train_features = custom_vgg_model.predict(train_generator, steps=n_train_samples, verbose=1)

np.savetxt("model2_svm/train_features.csv",  
           train_features, 
           delimiter =", ",  
           fmt ='% s') 

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    train_dir, 
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=1
)


filenames = test_generator.filenames
n_test_samples = len(filenames)
test_features = custom_vgg_model.predict(train_generator, steps=n_test_samples, verbose=1)

np.savetxt("model2_svm/test_features.csv",  
           train_features, 
           delimiter =", ",  
           fmt ='% s') 

Found 10278 validated image filenames belonging to 4 classes.
10278/10278 [==============================] - 919s 89ms/step
Found 1143 validated image filenames belonging to 4 classes.
1143/1143 [==============================] - 101s 88ms/step


In [52]:
X = train_features
y = np.array(train_generator.labels)

In [53]:
train_features.shape

(10278, 25088)

In [54]:
from sklearn import svm
lin_svm = svm.SVC(kernel='linear')
quadratic_svm = svm.SVC(kernel='poly', degree=2)
exp_svm = svm.SVC(kernel='rbf')
lin_svm.fit(X,y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
quadratic_svm.fit(X,y)

In [ ]:
exp_svm.fit(X,y)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(test_generator.labels, y_pred)
lin_score = lin_svm.score(test_features, test_generator.labels)
quadratic_score = quadratic_svm.score(test_features, test_generator.labels)
exp_score = exp_svm.score(test_features, test_generator.labels)

In [39]:
print("Lin score " + str(lin_score))
print("Quadratic score " + str(quadratic_score))
print("Exp score " + str(exp_score))

Lin score 0.3620689655172414
Quadratic score 0.3448275862068966
Exp score 0.3275862068965517


In [62]:
test_features

array([[0.4671375 , 0.        , 0.        , ..., 0.        , 0.3446469 ,
        0.        ],
       [0.27094406, 0.        , 0.24629644, ..., 0.        , 0.814126  ,
        0.        ],
       [0.8937061 , 0.        , 0.6163965 , ..., 0.        , 0.5872522 ,
        0.        ],
       [0.        , 0.        , 0.00475407, ..., 0.        , 0.6345401 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.986208  ,
        0.        ],
       [0.48684415, 0.        , 0.        , ..., 0.        , 0.70045   ,
        0.        ]], dtype=float32)

In [58]:
np.savetxt("model2_svm/lin_score.csv",  
           np.array([lin_score]), 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model2_svm/quadratic_score.csv",  
           quadratic_score, 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model2_svm/exp_score.csv",  
           exp_score, 
           delimiter =", ",  
           fmt ='% s') 

ValueError: Expected 1D or 2D array, got 0D array instead